---
title: Crossover analysis
description: Using xOPR to automatically find radar crossovers
date: 2025-09-09
---

In this notebook, we demonstratate how to automatically find and analyze radar crossovers, both within and between campaigns.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import shapely
import scipy.constants
import pandas as pd
import traceback
import geopandas as gpd
from tqdm import tqdm

import xopr.opr_access
import xopr.geometry

import holoviews as hv
import hvplot.xarray
import hvplot.pandas
hvplot.extension('bokeh')

In [ ]:
# Useful projections
epsg_3031 = ccrs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
latlng = ccrs.PlateCarree()
features = gf.ocean.options(scale='50m').opts(projection=epsg_3031) * gf.coastline.options(scale='50m').opts(projection=epsg_3031)

In [ ]:
# Establish an OPR session
# You'll probably want to set a cache directory if you're running this locally to speed
# up subsequent requests. You can do other things like customize the STAC API endpoint,
# but you shouldn't need to do that for most use cases.
opr = xopr.opr_access.OPRConnection(cache_dir="radar_cache")

# Or you can open a connection without a cache directory (for example, if you're parallelizing
# this on a cloud cluster without persistent storage).
#opr = xopr.OPRConnection()

In [ ]:
region = xopr.geometry.get_antarctic_regions(name='David', merge_regions=True, simplify_tolerance=100)

# Create a GeoViews object for the selected region
region_gv = gv.Polygons(region, crs=latlng).opts(
    color='green',
    line_color='black',
    fill_alpha=0.5,
    projection=epsg_3031,
)
features * region_gv

In [ ]:
stac_items = opr.query_frames(geometry=region)

# This feels sloppy, but there would be an easy fix if we just natively returned a GeoDataFrame from query_frames.
# It seems like it would generally be an improvement over a list and GeoPandas is already a dependency.

stac_items_df = gpd.GeoDataFrame(stac_items)
stac_items_df = stac_items_df.set_index('id')
stac_items_df = stac_items_df.set_geometry(stac_items_df['geometry'].apply(shapely.geometry.shape))
stac_items_df.crs = "EPSG:4326"
stac_items_df = stac_items_df.to_crs(epsg_3031)

print(f"Found {len(stac_items)} frames across {stac_items_df['collection'].nunique()} collections:")

stac_items_df.groupby('collection').size()

In [ ]:
len(opr.query_frames(seasons='2013_Antarctica_P3', flight_ids='20131120_01'))

In [ ]:
stac_items[0]['properties']

In [ ]:
# TODO TODO TODO

stac_item = stac_items[0]

layers = opr.get_layers_db(stac_item)
layers[2]

In [ ]:
flight_lines = stac_items_df.hvplot(by='collection')
(features * region_gv * flight_lines).opts(projection=epsg_3031, frame_width=600, aspect='equal')

In [ ]:
# Definitely room for improvement here, but the geopandas spatial join works pretty nicely for finding intersections between frames.
# Couple thoughts:
# 1. We might want to include a helper function for this
# 2. This will not find self-intersections within a single frame.

tmp_df = stac_items_df.reset_index()
tmp_df['geom'] = tmp_df.geometry
intersections = gpd.sjoin(tmp_df, tmp_df, how='inner', predicate='intersects', lsuffix='1', rsuffix='2')
intersections = intersections[intersections['id_1'] != intersections['id_2']]
intersections['intersection_geometry'] = intersections.apply(lambda row: row['geom_1'].intersection(row['geom_2']), axis=1)
intersections.set_geometry('intersection_geometry', inplace=True, crs=stac_items_df.crs)
intersections = intersections.drop_duplicates(subset=['intersection_geometry'])
intersections = intersections.explode(index_parts=True).reset_index(drop=True)
print(f"Found {len(intersections)} crossover points between flight lines.")
(features * region_gv * flight_lines * intersections.hvplot(label='Intersection Points', color='purple')).opts(frame_width=600, aspect='equal')

In [ ]:
# The result of this is a GeoDataFrame where every column from the intersecting frames is preserved, with suffixes _1 and _2 to distinguish them.
intersections.iloc[0]

In [ ]:
# This surfaced a lot of issues with layers...
# 1. It's annoying that you need to load a frame in order to get the layers for that flight. We should probably have a way to get layers directly from a STAC item.
# 1a. This work-around is OK for loading layer files, but it doesn't help with loading database layers. The databse layers include timing but not position information, so additional queries would be needed to get the position.
# 2. We should probably make both get_layers_files and get_layers_db return the same dataset structure.
# 3. Caching is a bit awkard here because get_layers_db will load the entire flight line.
# 4. The functionality to trim layers to the time range of the frame is broken in some cases because slow_time is sometimes returned as a float64 instead of a datetime64[ns]. We should fix this.

layer_cache = {}

def get_layers_stac_item(stac_item):
    ds_fake = xr.Dataset()
    ds_fake.attrs['season'] = stac_item['collection']
    ds_fake.attrs['segment'] = f"{stac_item['properties'].get('opr:date')}_{stac_item['properties'].get('opr:flight'):02d}"

    if (ds_fake.attrs['season'], ds_fake.attrs['segment']) in layer_cache:
        #print(f"Using cached layers for {ds_fake.attrs['season']} segment {ds_fake.attrs['segment']}...")
        return layer_cache[(ds_fake.attrs['season'], ds_fake.attrs['segment'])]

    #print(f"Loading layers for {ds_fake.attrs['season']} segment {ds_fake.attrs['segment']}...")

    layers = None
    layers = opr.get_layers_files(ds_fake)

    layer_cache[(ds_fake.attrs['season'], ds_fake.attrs['segment'])] = layers

    return layers

In [ ]:
layer_cache = {}

def get_basal_layer(stac_item):
    flight_id = f"{stac_item['properties'].get('opr:date')}_{stac_item['properties'].get('opr:flight'):02d}"
    if (stac_item['collection'], flight_id) in layer_cache:
        #print(f"Using cached basal layer for {flight_id}")
        return layer_cache[(stac_item['collection'], flight_id)]

    try:
        layers = opr.get_layers_files(stac_item)
        basal_layer = layers[2]
        #print(f"Loaded basal layer from files for {flight_id}")
        layer_cache[(stac_item['collection'], flight_id)] = basal_layer
        return basal_layer
    except Exception as e:
        layers = opr.get_layers_db(stac_item)
        basal_layer = layers[2]
        #print(f"Loaded basal layer from database for {flight_id}")
        layer_cache[(stac_item['collection'], flight_id)] = basal_layer
        return basal_layer

In [ ]:
intersections['elev_1'] = np.nan
intersections['elev_2'] = np.nan

idx = 0
row = intersections.iloc[idx]

for idx, row in tqdm(intersections.iterrows(), total=len(intersections)):
    stac_item_1 = stac_items_df.loc[row['id_1']].to_dict()
    stac_item_2 = stac_items_df.loc[row['id_2']].to_dict()

    try:
        bed_1 = get_basal_layer(stac_item_1).rename({'lat': 'Latitude', 'lon': 'Longitude'})
        bed_2 = get_basal_layer(stac_item_2).rename({'lat': 'Latitude', 'lon': 'Longitude'})

        bed_1 = xopr.geometry.project_dataset(bed_1, "EPSG:3031")
        bed_2 = xopr.geometry.project_dataset(bed_2, "EPSG:3031")

        x, y = row.intersection_geometry.coords[0]

        dist_1 = np.sqrt((bed_1['x'] - x)**2 + (bed_1['y'] - y)**2)
        dist_2 = np.sqrt((bed_2['x'] - x)**2 + (bed_2['y'] - y)**2)

        elev_1 = bed_1['elev'][(dist_1 == dist_1.min())].values[0].item()
        elev_2 = bed_2['elev'][(dist_2 == dist_2.min())].values[0].item()

        intersections.at[idx, 'elev_1'] = elev_1
        intersections.at[idx, 'elev_2'] = elev_2
    except Exception as e:
        print(f"Error processing intersection {idx} between {row['id_1']} and {row['id_2']}: {repr(e)}")
        #traceback.print_exc()

intersections['elev_diff'] = intersections['elev_1'] - intersections['elev_2']

In [ ]:
# I believe the fairly few intersections still in the set are due to the older flights not having layer files (or not having layer files with bed picks).
# We probably just need to get db layer data working for those.

intersections[~intersections['elev_diff'].isna()][['id_1', 'collection_1', 'id_2', 'collection_2', 'elev_1', 'elev_2', 'elev_diff']].head()

In [ ]:
intersections_success = intersections.dropna()
intersections_success['idx'] = intersections_success.index
hover_tooltips = [
    ("Index", "@idx"),
    ("Collection 1", "@collection_1"),
    ("Collection 2", "@collection_2"),
    ("Difference", "@elev_diff{0.00} m"),
]
hv_int = intersections_success.hvplot(color='elev_diff', cmap='coolwarm_r', hover_cols=['idx', 'collection_1', 'collection_2', 'elev_diff'], hover_tooltips=hover_tooltips)
hv_int = hv_int.opts(scalebar=True)
(features * region_gv * flight_lines * hv_int).opts(frame_width=600, aspect='equal', active_tools=['pan', 'wheel_zoom'])